In [1]:
dirPath = '/srv/home/whao/NextGen/new_debug/run/run3/datasets/balloon/train'

In [2]:
import csv
import matplotlib
import pandas as pd
import json
import os
import math
import re
from utils.postProcessing import *#img_ellipse_fitting
from utils.imageUtils import cropImage
from utils import imageUtils, postProcessing, DefectDetectionDataset, evaluation, visualization
import matplotlib.pyplot as plt

/srv/home/whao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
imagefolder = '/srv/home/whao/NextGen/new_debug/run/run3/datasets/datasets/Loop_detection/images/'
root = '/srv/home/whao/NextGen/new_debug/run/run3/datasets/datasets/Loop_detection'

files = os.listdir('/srv/home/whao/NextGen/new_debug/run/run3/datasets/balloon/train/')    
dataset = DefectDetectionDataset(data_dir = root, split = 'train')

All_Image_Defects_List = list()
for item in range(len(dataset)):
    img, bboxes, label = dataset[item]
    subimages, bboxes = cropImage(img, bboxes)
    current_img_defect_List = list()
    for subim, bbox in zip(subimages, bboxes):
        defects_Dict = dict()
        defects_X_List = list()
        defects_Y_List = list()
        region1 = flood_fitting(subim)
        result = (int(region1['centroid'][0]+bbox[0]), int(region1['centroid'][1]+bbox[1]),
                    int(region1['minor_axis_length'] / 2), int(region1['major_axis_length'] / 2),
                    -region1['orientation'])
        rr,cc = draw.ellipse_perimeter(*result)
        # rr is row which is x in pixel
        # cc is column which is y in pixel
        # (rr,cc) represents one data points and there may be redundent data points
        assert len(rr) == len(cc)
        flag = 0
        for i in range(len(cc)):
            defects_X_List.append(cc[i])
            defects_Y_List.append(rr[i])
            if ((cc[i] > 1024 or rr[i] > 1024) and flag == 0):
                print(files[item])
                flag = 1
        # add defect's x,y corrdinates into the dictionary
        # update dictionary
        defects_Dict['X'] = defects_X_List
        defects_Dict['Y'] = defects_Y_List
        current_img_defect_List.append(defects_Dict)
    All_Image_Defects_List.append(current_img_defect_List)

FileNotFoundError: [Errno 2] No such file or directory: '/srv/home/whao/NextGen/new_debug/run/run3/datasets/datasets/Loop_detection/trainimages.txt'